In [1]:
import numpy as np
import pandas as pd
import camelot as cmt
import re
import numpy_financial as np_fin
from sklearn.preprocessing import MinMaxScaler as MMS
import tabula as tbl
# https://towardsdatascience.com/scraping-table-data-from-pdf-files-using-a-single-line-in-python-8607880c750

## Later: Calculate the nearest five to the recommendation on either side then list 
## for this we use either nearest 5 to the candidate_row or calculate the difference score for each column, as is done in this nb
## The reason we want to have absolute differences is that it closely resembles what the user wanted. (we don't know if higher is preferred or even better)

In [2]:
invest_pat = r"C:\Users\bseot\Documents\EXOODA\INVESTEC\91-IP-TFSA-List-of-Funds-en.pdf"

In [3]:
df_invest= cmt.read_pdf(
    invest_pat,
    pages='1-8',
    password=None,
    flavor='stream',
    suppress_stdout=False,
#     layout_kwargs={},
#     **kwargs,
)

In [4]:
invest_column_names = ['FUND NAME', 'CLASS', 'TOTAL ANNUAL FEE', 'FOREIGN EXPOSURE', 'EQUITY EXPOSURE', 'PROPERTY EXPOSURE', 'Annual Growth Rate']
df_large_invest = df_invest[2].df.drop([0,1,2])
for table in df_invest[3:]:
    tab = table.df
    tab = tab.drop([0,1,2], axis = 0)
    df_large_invest = pd.concat([df_large_invest,tab],axis = 0)
np.random.seed(97)
df_large_invest['Annual Growth Rate'] = np.random.normal(loc = 0.06, scale = 0.02, size = len(df_large_invest))
df_large_invest.columns = invest_column_names

In [5]:
df_large_invest = df_large_invest.reset_index()
df_large_invest = df_large_invest.drop('index', axis = 1)
df_large_invest = df_large_invest.drop('CLASS', axis = 1)
df_large_invest = df_large_invest[df_large_invest['TOTAL ANNUAL FEE'] != '']

In [6]:
df_large_invest_crude = df_large_invest.copy()
df_large_invest_crude = df_large_invest_crude.reset_index(drop = True)
cols = df_large_invest.drop('FUND NAME', axis = 1).columns
df_large_invest[cols[:-1]] = df_large_invest[cols[:-1]].applymap(lambda x: x.replace('%',''))
df_large_invest[cols] = df_large_invest[cols].apply(pd.to_numeric, errors='coerce')
df_large_invest = df_large_invest.reset_index(drop = True)

In [7]:
df_large_invest_crude

,FUND NAME,TOTAL ANNUAL FEE,FOREIGN EXPOSURE,EQUITY EXPOSURE,PROPERTY EXPOSURE,Annual Growth Rate
0,Investec Cautious Managed Fund,0.85%,25.00%,40.00%,15.00%,0.051408
1,Investec Diversified Income Fund,0.60%,25.00%,3.00%,5.00%,0.045072
2,Investec Equity Fund,1.00%,25.00%,100.00%,10.00%,0.080137
3,Investec Global Franchise FF,1.10%,100.00%,100.00%,0.00%,0.048178
4,Investec Global Multi Asset Income FF,1.10%,100.00%,40.00%,10.00%,0.037453
...,...,...,...,...,...,...
130,STANLIB Global Balanced Feeder Fund,0.50%,100%,75%,0%,0.062331
131,STANLIB Global Property Feeder Fund,0.40%,100%,0%,100%,0.043593
132,STANLIB Income Fund,0.60%,0.00%,0.00%,0.00%,0.083120
133,STANLIB Property Income Fund,1.00%,0.00%,0.00%,100.00%,0.074993


In [8]:
df_large_invest['TOTAL ANNUAL FEE'] = (df_large_invest['TOTAL ANNUAL FEE']-100)*-1

# Ranking

In [9]:
df_large_invest['Annual Growth Rate (%)'] = df_large_invest['Annual Growth Rate']
# df_large_invest = df_large_invest.drop('Annual Growth Rate', axis = 1)
df_large_invest[cols] = MMS().fit_transform(df_large_invest[cols])
df_large_invest[cols] = df_large_invest[cols]*100
df_large_invest[cols] = round(df_large_invest[cols], 2)

In [10]:
df_large_invest[cols] = round(df_large_invest[cols].rank(numeric_only=True, axis = 1, method = 'min')).astype(int)

In [11]:
df_large_invest['EScore'] = df_large_invest[cols].sum(axis = 1)

In [12]:
fee = input('On a scale of 1-10, how important are fees to you?')
fee = int(round(int(fee)/2, 0))

fex = input('How much foreign exposure would you like the investment to have ( in percentage terms)?')
fex = int(round(int(fex)/2, 0)/10)

eqx = input('How much equity exposure would you like the investment to have ( in percentage terms)?')
eqx = int(round(int(eqx)/2, 0)/10)

prx = input('How much property exposure would you like the investment to have ( in percentage terms)?')
prx = int(round(int(prx)/2, 0)/10)

# #for now, we just ask how nb but in future we will check the initial and end

# rat = input('What growth rate would you like your investment to have (Based on Historical performance, the highest was 10% per annum)?')
# rat = int(round(int(rat)/2, 0))

# # To Do: annuity
princ = input('What is your initial deposit amount?')
princ = int(princ)
insta = input('What will your monthly installment be?')
insta = int(insta)

n =input('For how many years would you like to keep this investment?')
n = int(n)

mat = input('To what amount would you like your investment to grow?')
mat = int(mat)

On a scale of 1-10, how important are fees to you?0
How much foreign exposure would you like the investment to have ( in percentage terms)?70
How much equity exposure would you like the investment to have ( in percentage terms)?78
How much property exposure would you like the investment to have ( in percentage terms)?0
What is your initial deposit amount?1000000
What will your monthly installment be?3000
For how many years would you like to keep this investment?4
To what amount would you like your investment to grow?1450000


In [13]:
irate = np_fin.rate(nper = n*12, pmt = insta, pv = princ, fv = -mat, when='end', guess=None, tol=None, maxiter=100)

In [26]:
irate = irate*12

# Make a copy for each user then determine the best result from their difference (cell block will have a difference calculated)

In [15]:
ins = [fee, fex, eqx, prx, irate]
diff_cols = ['TOTAL ANNUAL FEE', 'FOREIGN EXPOSURE', 'EQUITY EXPOSURE', 'PROPERTY EXPOSURE', 'Annual Growth Rate (%)']

user_diffs = df_large_invest.copy()
for i, col_name in enumerate(diff_cols):
    user_diffs[col_name] = np.abs((user_diffs[col_name] - ins[i]))
    
user_diffs['EScore'] = user_diffs[diff_cols].sum(axis = 1)
user_diffs['EScore'] = np.abs(21 - user_diffs['EScore'])

In [16]:
user_diffs

,FUND NAME,TOTAL ANNUAL FEE,FOREIGN EXPOSURE,EQUITY EXPOSURE,PROPERTY EXPOSURE,Annual Growth Rate,Annual Growth Rate (%),EScore
0,Investec Cautious Managed Fund,5,1,0,1,4,0.046143,13.953857
1,Investec Diversified Income Fund,5,0,2,2,4,0.039807,11.960193
2,Investec Equity Fund,3,1,2,1,4,0.074872,13.925128
3,Investec Global Franchise FF,2,1,1,1,3,0.042913,15.957087
4,Investec Global Multi Asset Income FF,2,2,1,1,3,0.032189,14.967811
...,...,...,...,...,...,...,...,...
130,STANLIB Global Balanced Feeder Fund,3,2,1,1,2,0.057066,13.942934
131,STANLIB Global Property Feeder Fund,3,1,2,4,2,0.038328,10.961672
132,STANLIB Income Fund,4,2,2,1,5,0.077855,11.922145
133,STANLIB Property Income Fund,3,2,2,5,4,0.069728,8.930272


In [18]:
user_final = user_diffs.sort_values(by = 'EScore', ascending = False).iloc[:6,:]

In [20]:
df_large_invest_crude['EScore'] = user_diffs['EScore']
df_large_invest_crude.iloc[user_final.index.values]

,FUND NAME,TOTAL ANNUAL FEE,FOREIGN EXPOSURE,EQUITY EXPOSURE,PROPERTY EXPOSURE,Annual Growth Rate,EScore
16,Foord Equity Fund,1.35%,20.00%,100.00%,0.00%,0.036020,16.969245
119,Satrix MSCI World Equity Index FF,0.20%,100.00%,100.00%,0.00%,0.046235,16.959030
104,Megafin SCI Absolute Fund of Funds,1.13%,70.00%,75.00%,25.00%,0.081852,16.923413
12,Investec Worldwide Flexible Fund,1.10%,100.00%,100.00%,25.00%,0.045602,15.959663
5,Investec Global Strategic Equity Feeder Fund,1.10%,100.00%,100.00%,25.00%,0.046361,15.958903
3,Investec Global Franchise FF,1.10%,100.00%,100.00%,0.00%,0.048178,15.957087


In [28]:
df_large_invest_crude.to_dict('records')

[{'FUND NAME': 'Investec Cautious Managed Fund',
  'TOTAL ANNUAL FEE': '0.85%',
  'FOREIGN EXPOSURE': '25.00%',
  'EQUITY EXPOSURE': '40.00%',
  'PROPERTY EXPOSURE': '15.00%',
  'Annual Growth Rate': 0.051408032919020755,
  'EScore': 13.953856755661786},
 {'FUND NAME': 'Investec Diversified Income Fund',
  'TOTAL ANNUAL FEE': '0.60%',
  'FOREIGN EXPOSURE': '25.00%',
  'EQUITY EXPOSURE': '3.00%',
  'PROPERTY EXPOSURE': '5.00%',
  'Annual Growth Rate': 0.04507221622845459,
  'EScore': 11.96019257235235},
 {'FUND NAME': 'Investec Equity Fund',
  'TOTAL ANNUAL FEE': '1.00%',
  'FOREIGN EXPOSURE': '25.00%',
  'EQUITY EXPOSURE': '100.00%',
  'PROPERTY EXPOSURE': '10.00%',
  'Annual Growth Rate': 0.08013695276646879,
  'EScore': 13.925127835814337},
 {'FUND NAME': 'Investec Global Franchise FF',
  'TOTAL ANNUAL FEE': '1.10%',
  'FOREIGN EXPOSURE': '100.00%',
  'EQUITY EXPOSURE': '100.00%',
  'PROPERTY EXPOSURE': '0.00%',
  'Annual Growth Rate': 0.04817752266201059,
  'EScore': 15.957087265918

What is the future value after 10 years of saving $200 now, with an additional monthly savings of $200. Assume the interest rate is 6% (annually) compounded monthly?

In [27]:
# TOTALANNUALFEE = df_large_invest.sort_values(by = 'TOTAL ANNUAL FEE', ascending = False)[['FUND NAME','TOTAL ANNUAL FEE']]
# FOREIGNEXPOSURE = df_large_invest.sort_values(by = 'FOREIGN EXPOSURE', ascending = False)[['FUND NAME','FOREIGN EXPOSURE']]
# EQUITYEXPOSURE = df_large_invest.sort_values(by = 'EQUITY EXPOSURE', ascending = False)[['FUND NAME','EQUITY EXPOSURE']]
# PROPERTYEXPOSURE = df_large_invest.sort_values(by = 'PROPERTY EXPOSURE', ascending = False)[['FUND NAME','PROPERTY EXPOSURE']]

# Run app. Main app is exooda_invest.py

In [25]:
import dash_core_components as dcc
import dash_html_components as html
import dash
from dash.dependencies import Input, Output

app = dash.Dash(__name__)
app.layout = html.Div(
    [
        dcc.Input(id="dfalse", type="number", placeholder="Debounce False"),
        dcc.Input(
            id="dtrue", type="number",
            debounce=True, placeholder="Debounce True",
        ),
        dcc.Input(
            id="input_range", type="number", placeholder="input with range",
            min=10, max=100, step=3,
        ),
        html.Hr(),
        html.Div(id="number-out"),
    ]
)


@app.callback(
    Output("number-out", "children"),
    [Input("dfalse", "value"), Input("dtrue", "value"), Input("input_range", "value")],
)
def number_render(fval, tval, rangeval):
    return "dfalse: {}, dtrue: {}, range: {}".format(fval, tval, rangeval)


if __name__ == "__main__":
    app.run_server(debug=True)

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


SystemExit: 1

C:\Users\bseot\anaconda3\envs\exoodaEnv\lib\site-packages\IPython\core\interactiveshell.py:3426: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
